In [1]:
import pandas as pd
import numpy as np

In [2]:
# Please change the file location as needed
file_location = "/home/yamunad/AICamp/AWS_Full_Stack-03-11/Session2/FE/bank_churn_project_1.csv"
data = pd.read_csv(file_location)
data

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,42,2,0.00,1,1,1,101348.88,1
1,608,41,1,83807.86,1,0,1,112542.58,0
2,502,42,8,159660.80,3,1,0,113931.57,1
3,699,39,1,0.00,2,0,0,93826.63,0
4,850,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0
9996,516,35,10,57369.61,1,1,1,101699.77,0
9997,709,36,7,0.00,1,0,1,42085.58,1
9998,772,42,3,75075.31,2,1,0,92888.52,1


In [3]:
# Please change the label to match dataset
label = 'Exited'

In [4]:
# Rearrage the dataset columns
cols = data.columns.tolist()
cols

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Exited']

In [5]:
colIdx = data.columns.get_loc(label)
# Do nothing if the label is in the 0th position
# Otherwise, change the order of columns to move label to 0th position
if colIdx != 0:
    cols = cols[colIdx:colIdx+1] + cols[0:colIdx] + cols[colIdx+1:]
cols

['Exited',
 'CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [6]:
# Change the order of data so that label is in the 0th column
modified_data = data[cols]
modified_data

,Exited,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,1,619,42,2,0.00,1,1,1,101348.88
1,0,608,41,1,83807.86,1,0,1,112542.58
2,1,502,42,8,159660.80,3,1,0,113931.57
3,0,699,39,1,0.00,2,0,0,93826.63
4,0,850,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...
9995,0,771,39,5,0.00,2,1,0,96270.64
9996,0,516,35,10,57369.61,1,1,1,101699.77
9997,1,709,36,7,0.00,1,0,1,42085.58
9998,1,772,42,3,75075.31,2,1,0,92888.52


In [7]:
# Remove the useless columns
# Note that below code is a sample
# Please change the value as you see fit

# Maximum categories allowed in a column
# If a column contains more than 10 categories, it is dropped
MAX_CAT_ALLOWED = 10

cat_cols = modified_data.select_dtypes(exclude=['int', 'float']).columns
cat_cols = set(cat_cols) - {label}
#cat_cols

In [8]:
useless_cols = []
for cat_column_features in cat_cols:
    num_cat = modified_data[cat_column_features].nunique()
    if num_cat > MAX_CAT_ALLOWED:
        useless_cols.append(cat_column_features)

In [9]:
# If a column contains only 1 catetgory, it is dropped
for feature_column in modified_data.columns:
    num_cat = modified_data[feature_column].nunique()
    if num_cat <= 1:
        useless_cols.append(feature_column)

In [10]:
modified_data = modified_data.drop(useless_cols, axis=1)
#modified_data

In [11]:
# One hot encode and fill missing values
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [12]:
# Remove label so that it is not encoded
data_without_label = modified_data.drop([label], axis=1)
#data_without_label

In [13]:
# Fills missing values with the median value
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))])


In [14]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [15]:
numeric_features = data_without_label.select_dtypes(include=['int64',
                                                    'float64']).columns
#numeric_features

In [16]:
categorical_features = data_without_label.select_dtypes(exclude=['int64',
                                                            'float64']).columns
#categorical_features

In [17]:
# Create the column transformer
preprocessor_cols = ColumnTransformer(
    transformers=[('num', numeric_transformer, numeric_features),
                  ('cat', categorical_transformer, categorical_features)])

In [18]:
# Create a pipeline with the column transformer, note that
# more things can be added to this pipeline in the future
preprocessor = Pipeline(steps=[('preprocessor', preprocessor_cols)])
preprocessor.fit(data_without_label)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),


In [19]:
modified_data_without_label = preprocessor.transform(data_without_label)
modified_data_without_label

array([[6.1900000e+02, 4.2000000e+01, 2.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [6.0800000e+02, 4.1000000e+01, 1.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [5.0200000e+02, 4.2000000e+01, 8.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [7.0900000e+02, 3.6000000e+01, 7.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [7.7200000e+02, 4.2000000e+01, 3.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [7.9200000e+02, 2.8000000e+01, 4.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [20]:
if (type(modified_data_without_label) is not np.ndarray):
    modified_data_without_label = modified_data_without_label.toarray()

In [21]:
modified_data_array = np.concatenate(
    (np.array(modified_data[label]).reshape(-1, 1),
     modified_data_without_label), axis=1)
#modified_data_array

In [22]:
# Save the processed file, please change preicison in fmt as needed
np.savetxt("data_processed.csv", modified_data_array, delimiter=",", fmt='%1.3f')

In [23]:
# Split the file into train and test (80% train and 20% test)
from sklearn.model_selection import train_test_split
train, test= train_test_split(modified_data_array, test_size=0.2)


In [24]:
# Save the train file, please change preicison in fmt as needed
np.savetxt("train.csv", train, delimiter=",", fmt='%1.3f')

# Save the test file, please change preicison in fmt as needed
np.savetxt("test.csv", test, delimiter=",", fmt='%1.3f')